<a href="https://colab.research.google.com/github/goransavich/cnn-xgboost/blob/main/cnnxgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#cnn with xgboost
import tensorflow as tf
import numpy as np
from matplotlib import pyplot
np.random.seed(1337)  # for reproducibility
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Flatten, Input, Dropout
#from tensorflow.keras.optimizers import Adam
#xgboost
import xgboost as xgb
import pandas as pd
from sklearn import metrics

In [15]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd

train_name = list(uploaded.keys())[0]

train_data = pd.read_csv(io.BytesIO(uploaded[train_name]))

Saving Coffee_TRAIN.csv to Coffee_TRAIN.csv


In [16]:
train_data.shape

(28, 287)

In [17]:
x_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1:]

In [18]:
print('Shape trening podataka:' + str(x_train.shape))
print('Shape trening y' + str(y_train.shape))

Shape trening podataka:(28, 286)
Shape trening y(28, 1)


In [19]:
classes_in_dataset = len(np.unique(y_train))

In [20]:
classes_in_dataset

2

In [21]:
uploaded = files.upload()

test_name = list(uploaded.keys())[0]

test_data = pd.read_csv(io.BytesIO(uploaded[test_name]))

Saving Coffee_TEST.csv to Coffee_TEST.csv


In [22]:
x_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1:]
print('Shape test podataka:' + str(x_test.shape))
print('Shape test klase' + str(y_test.shape))

Shape test podataka:(28, 286)
Shape test klase(28, 1)


In [23]:
print(len(x_train.shape))

2


In [24]:
if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x_train = x_train.values.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]

In [25]:
print(input_shape)

(286, 1)


In [26]:
## uraditi one hot encoding izlaznih labela######
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()


import sklearn
# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
#creating a model
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
inputs = input_shape
input_layer = Input(inputs)
conv2d1 = Convolution1D(filters=32, kernel_size=3, padding='valid')(input_layer)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling1D(pool_size=2, strides=2, padding='valid')(activation1)
dropout1 = Dropout(0.5)(maxpooling1)

conv2d2 = Convolution1D(filters=64, kernel_size=3, padding='valid')(dropout1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling1D(pool_size=2, strides=2, padding='valid')(activation2)
#dropout2 = Dropout(0.5)(maxpooling2)

flatten = Flatten()(maxpooling2)
#output_for_xgb = Model(inputs = input_layer, outputs = flatten)
dense1 = Dense(100)(flatten)
activation3 = Activation('relu')(dense1)

#dense2 = Dense(units=classes_in_dataset)(activation3)
#activation4 = Activation('softmax')(dense2)

output_for_xgboost = Model(inputs=input_layer, outputs=flatten)

#model = Model(inputs=input_layer, outputs=activation4)


#adam = Adam(learning_rate=1e-4)

# We add metrics to get more results you want to see
#model.compile(optimizer='Adam',loss='mean_squared_error',metrics=['accuracy'])

#print('Training ------------')
# Another way to train the model
#history = model.fit(x_train, y_train, epochs=40, batch_size=16)

#print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
#loss, accuracy = model.evaluate(x_test, y_test)

#print('\ntest loss: ', loss)
#print('\ntest accuracy: ', accuracy)

#pyplot.plot(history.history['mse'])
#pyplot.plot(history.history['accuracy'])
#pyplot.show()

######### XGBOOST #############
x_train_xg = output_for_xgboost.predict(x_train)
x_test_xg = output_for_xgboost.predict(x_test)

train_x = x_train_xg
test_x = x_test_xg
#print(x_train_xg.shape)
#print(x_test_xg.shape)

dataset = xgb.DMatrix(train_x, label=y_train)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class': classes_in_dataset,'objective':'multi:softmax' } 
model_xg = xgb.train(params, dataset, num_boost_round=150, evals=watchlist)


test_x = xgb.DMatrix(test_x)
result = model_xg.predict(test_x)
#print(t)
#m = LogisticRegression()
#m = SVC(kernel='rbf',decision_function_shape='ovr')
#m.fit(X,y)
#print(m)

#print(metrics.classification_report(y_test, result))
#print(metrics.confusion_matrix(y_test, result))


#d_train = xgb.DMatrix(x_train_xgb, label=y_train)
#d_test = xgb.DMatrix(x_test, label=y_test)

#param = {
#    'eta': 0.3, 
#    'max_depth': 3,  
#    'objective': 'multi:softprob',  
#    'num_class': classes_in_dataset} 

#steps = 20  # The number of training iterations
#model = xgb.train(param, d_train, steps)

from sklearn.metrics import precision_score, recall_score, accuracy_score

#preds = model.predict(d_test)
best_preds = np.asarray([np.argmax(line) for line in result])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))



[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[0]	train-merror:0.071429
[1]	train-merror:0.035714
[2]	train-merror:0.035714
[3]	train-merror:0.035714
[4]	train-merror:0
[5]	train-merror:0
[6]	train-merror:0
[7]	train-merror:0
[8]	train-merror:0
[9]	train-merror:0
[10]	train-merror:0
[11]	train-merror:0
[12]	train-merror:0
[13]	train-merror:0
[14]	train-merror:0
[15]	train-merror:0
[16]	train-merror:0
[17]	train-merror:0
[18]	train-merror:0
[19]	train-merror:0
[20]	train-merror:0
[21]	train-merror:0
[22]	train-merror:0
[23]	train-merror:0
[24]	train-merror:0
[25]	train-merror:0
[26]	train-merror:0
[27]	train-merror:0
[28]	train-merror:0
[29]	train-merror:0
[30]	train-merror:0
[31]	train-merror:0
[32]	train-merror:0
[33]	train-merror:0
[34]	train-merror:0


ValueError: ignored